[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyMPDATA.git/main?urlpath=lab/tree/examples/PyMPDATA_examples/advection_diffusion_2d/advection-diffusion-2d.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/examples/blob/main/examples/PyMPDATA_examples/advection_diffusion_2d/advection-diffusion-2d.ipynb)

## common settings

In [ ]:
SETUP = {
    "nx": 32,
    "ny": 32,
    "ux": 0.25,
    "uy": 0.25,
    "dt": 0.025,
    "tmax": 5.0,
    "polydeg": 3
}

assert SETUP["nx"] == SETUP["ny"]

import json
with open('setup.json', 'w', encoding='UTF-8') as f:
    json.dump(SETUP, f)

## Trixi.jl

In [ ]:
%%writefile trixi.jl
import Pkg
Pkg.add(["JSON", "Trixi", "OrdinaryDiffEq", "Trixi2Vtk"])
using JSON
using Trixi
using OrdinaryDiffEq
using Trixi2Vtk

setup = JSON.parsefile("./setup.json")

advection_velocity = (setup["ux"], setup["uy"])
equations = LinearScalarAdvectionEquation2D(advection_velocity)
solver = DGSEM(polydeg = setup["polydeg"])

function initial_condition(x, t, equations::LinearScalarAdvectionEquation2D)
    return SVector(sin(pi*sum(x)) + 1)
end

cells_per_dimension = (setup["nx"], setup["ny"])
coordinates_min = (-1.0, -1.0)
coordinates_max = ( 1.0,  1.0)

mesh = StructuredMesh(cells_per_dimension, coordinates_min, coordinates_max)
semi = SemidiscretizationHyperbolic(mesh, equations, initial_condition, solver)

tspan = (0.0, setup["tmax"])
ode = semidiscretize(semi, tspan);

summary_callback = SummaryCallback()
save_solution = SaveSolutionCallback(interval=100)

stepsize_callback = StepsizeCallback(cfl = 1.6)

callbacks = CallbackSet(summary_callback, save_solution, stepsize_callback)

time_int_tol = 1e-6
sol = solve(ode, CarpenterKennedy2N54();
            abstol = time_int_tol,
            reltol = time_int_tol,
            dt = setup["dt"],
            ode_default_options()..., callback = callbacks);

summary_callback()
trixi2vtk("out/solution_000101.h5")

In [ ]:
_ = !julia trixi.jl 

In [ ]:
with open("solution_000101.vtu") as f:
    pass

## PyMPDATA

In [ ]:
import numpy as np
import meshio
from open_atmos_jupyter_utils import show_plot
import matplotlib.pyplot as plt
from PyMPDATA import Solver, ScalarField, VectorField, Stepper, Options
from PyMPDATA.boundary_conditions import Periodic

In [ ]:
mu = 0.0
dt = SETUP["dt"]
tmax = SETUP["tmax"]
nt = int(tmax / dt)

nx = SETUP["nx"] * SETUP["polydeg"] + 1
ny = SETUP["ny"] * SETUP["polydeg"] + 1
ux  = SETUP["ux"]
uy = SETUP["uy"]

omega = np.pi

min_x, min_y = -1, -1
max_x, max_y = 1, 1
dx_temp = (max_x - min_x) / (nx - 1)
dy_temp = (max_y - min_y) / (ny - 1)
min_x, max_x = min_x - dx_temp/2, max_x + dx_temp/2
min_y, max_y = min_y - dy_temp/2, max_y + dy_temp/2
dx = (max_x - min_x) / nx
dy = (max_y - min_y) / ny
Cx = ux * dt / dx
Cy = uy * dt / dy

In [ ]:
opt = Options(n_iters=3, non_zero_mu_coeff=True)
boundary_conditions = (Periodic(), Periodic())

In [ ]:
def analytic_solution(x, y, t):
    return np.sin(omega*(x-ux*t+y-uy*t))*np.exp(-2*mu*t*omega**2) + 1

In [ ]:
def z(t):
    return np.array(
    [
        analytic_solution(x, y, t=t) for x in np.linspace(min_x, max_x, nx)
        for y in np.linspace(min_y, max_y, ny)
    ],
    dtype=float
).reshape((nx, ny))

advectee = ScalarField(data=z(t=0), halo=opt.n_halo, boundary_conditions=boundary_conditions)

In [ ]:
field_x = np.full((nx+1, ny), Cx, dtype=opt.dtype)
field_y = np.full((nx, ny+1), Cy, dtype=opt.dtype)

advector = VectorField(
    data=(field_x, field_y),
    halo=opt.n_halo,
    boundary_conditions=(boundary_conditions[0], Periodic())
)

In [ ]:
stepper = Stepper(options=opt, n_dims=2)
solver = Solver(stepper=stepper, advector=advector, advectee=advectee)

In [ ]:
vmin = np.min(solver.advectee.get())
vmax = np.max(solver.advectee.get())

In [ ]:
_ = solver.advance(n_steps=nt, mu_coeff=(mu, mu))

In [ ]:
pympdata_result_state = solver.advectee.get().copy()

In [ ]:
plt.imshow(pympdata_result_state, cmap='viridis', vmin=vmin, vmax=vmax)
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')
plt.title('PyMDATA solution')
show_plot()

In [ ]:
mesh = meshio.read("solution_000101.vtu")
trixi_points = ((mesh.points[:,:2] + 1)*SETUP["nx"]*SETUP["polydeg"]/2).round().astype(np.int16)
assert trixi_points.shape[0] == SETUP["nx"]**2 * (SETUP["polydeg"] + 1)**2

In [ ]:
trixi_output = np.zeros(pympdata_result_state.shape)
for i in range(trixi_points.shape[0]):
    trixi_output[trixi_points[i][0], trixi_points[i][1]] = mesh.point_data['scalar'][i][0]

In [ ]:
plt.imshow(trixi_output, cmap='viridis', vmin=vmin, vmax=vmax)
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')
plt.title("Trixi solution")
show_plot()

In [ ]:
residual = pympdata_result_state - trixi_output
rmse = np.sqrt(np.mean(residual))
mae = np.mean(residual)
max_diff = np.max(np.abs(residual))
min_diff = np.min(np.abs(residual))

In [ ]:
assert np.allclose(rmse, 1.03e-2, 0.01)
assert np.allclose(mae, 1.06e-4, 0.01)
assert np.allclose(max_diff, 0.189, 0.01)
assert np.allclose(min_diff, 7.74e-5, 0.01)